In [27]:
import torch
import torch
import sys
import os
# sys.path.insert(0, os.path.abspath('../src/pretrained/MedViT'))
sys.path.insert(0, os.path.abspath('../'))


from src.pretrained.MedViT import MedViT 
# from MedViT import MedViT_small as tiny
                                       
model_pth = '../saved_models/medvit_mnist__base_wt75.pt'


In [28]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(device)

cpu


In [29]:
import os
print(os.getcwd())

c:\Users\rrahman3\PhDDissertation\EUFairGAN\notebooks


In [30]:
model = MedViT.MedViT_small()
model.proj_head[0] = torch.nn.Linear(in_features=1024, out_features=14, bias=True)
model.to(device)

initialize_weights...


MedViT(
  (stem): Sequential(
    (0): ConvBNReLU(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (1): ConvBNReLU(
      (conv): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (2): ConvBNReLU(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (3): ConvBNReLU(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=Tru

In [5]:
model.load_model(model_pth)
model.to(device)

/Users/nuha/Documents/UFairGAN/EUFairGAN/src/models/base_model.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(path, map_location=torch.

Model loaded from ../saved_models/medvit_mnist__base_wt75.pt


MedViT(
  (stem): Sequential(
    (0): ConvBNReLU(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (1): ConvBNReLU(
      (conv): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (2): ConvBNReLU(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (3): ConvBNReLU(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=Tru

In [17]:
from src.dataloader.medical_dataset import NIHChestXrayDataset
from torch.utils.data import DataLoader
dataset = NIHChestXrayDataset(
            metadata_file="../data/nihcc_chest_xray/nihcc_chest_xray_testing_samples.csv",
            image_dir="../data/nihcc_chest_xray/xray_images/",
            isTest = True
        )
dataloader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=True,
    num_workers=0
)

Index(['Unnamed: 0', 'id', 'Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration',
       'Mass', 'Nodule', 'Pleural Thickening', 'Pneumonia', 'Pneumothorax',
       'Pneumoperitoneum', 'Pneumomediastinum', 'Subcutaneous Emphysema',
       'Tortuous Aorta', 'Calcification of the Aorta', 'No Finding', 'subj_id',
       'Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width',
       'Height]', 'OriginalImagePixelSpacing[x', 'y]'],
      dtype='object')
21081


In [18]:

with torch.no_grad():
    for image, _, labels in dataloader:
        image, labels = image.to(device), labels.to(device)
        predictions = model(image)
        print(labels.shape)
        break

torch.Size([16, 14])


In [19]:
pred_sigmoid = torch.sigmoid(predictions)
y_pred_binary = (pred_sigmoid > 0.5).float()

In [22]:
idx = 3
print(pred_sigmoid[idx])
print(y_pred_binary[idx])
print(labels[idx])

tensor([2.1624e-02, 6.2502e-05, 7.5915e-05, 3.2518e-01, 3.2663e-04, 7.3744e-02,
        1.5572e-03, 2.2721e-05, 1.2167e-02, 2.9708e-02, 1.3481e-04, 1.8793e-05,
        7.2437e-04, 1.5246e-03], device='mps:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='mps:0')
tensor([0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       device='mps:0')


In [23]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, confusion_matrix

In [29]:
print(labels_binary)
print(pred_binary)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [30]:
pred_binary = (pred_sigmoid.detach().cpu().numpy() > 0.5).astype(int)
labels_binary = labels.detach().cpu().numpy()

accuracy = accuracy_score(labels_binary, pred_binary)
accuracy

0.625

In [25]:
from sklearn.metrics import confusion_matrix

# Assuming labels_binary and pred_binary are multilabel arrays
for i in range(labels_binary.shape[1]):
    y_true_label = labels_binary[:, i]
    y_pred_label = pred_binary[:, i]
    
    cm = confusion_matrix(y_true_label, y_pred_label)
    print(f"Confusion matrix for label {i}:")
    print(cm)

Confusion matrix for label 0:
[[15  0]
 [ 1  0]]
Confusion matrix for label 1:
[[15  0]
 [ 1  0]]
Confusion matrix for label 2:
[[13  0]
 [ 1  2]]
Confusion matrix for label 3:
[[13  1]
 [ 1  1]]
Confusion matrix for label 4:
[[15  1]
 [ 0  0]]
Confusion matrix for label 5:
[[16]]
Confusion matrix for label 6:
[[16]]
Confusion matrix for label 7:
[[14  0]
 [ 2  0]]
Confusion matrix for label 8:
[[14  0]
 [ 2  0]]
Confusion matrix for label 9:
[[15  0]
 [ 1  0]]
Confusion matrix for label 10:
[[15  0]
 [ 1  0]]
Confusion matrix for label 11:
[[16]]
Confusion matrix for label 12:
[[14  0]
 [ 1  1]]
Confusion matrix for label 13:
[[15  0]
 [ 1  0]]


/Users/nuha/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/Users/nuha/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/Users/nuha/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [26]:
confusion_matrix(labels_binary, pred_binary)

ValueError: multilabel-indicator is not supported

In [27]:
f1_score(labels_binary, pred_binary, average='micro')

0.36363636363636365

In [28]:
import numpy as np
if len(np.unique(labels_binary)) == 2:
    auc = roc_auc_score(labels_binary, pred_sigmoid.detach().cpu().numpy(), multi_class='ovo')
    print(f"ROC AUC: {auc}")
else:
    print("Only one class present in y_true. Skipping AUC calculation.")



ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [22]:
roc_auc_score(labels.detach().cpu().numpy(), pred_sigmoid.detach().cpu().numpy(), multi_class='ovo')

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [5]:
import torch 
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm 

In [6]:
# define loss function and optimizer
# if task == "multi-label, binary-class":
#     criterion = nn.BCEWithLogitsLoss()
# else:
#     criterion = nn.CrossEntropyLoss()

lr = 0.001
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [7]:
from src.dataloader.medical_dataset import NIHChestXrayDataset
from torch.utils.data import DataLoader
train_dataset = NIHChestXrayDataset(
            metadata_file="../data/nihcc_chest_xray/nihcc_chest_xray_training_samples.csv",
            image_dir="../data/nihcc_chest_xray/xray_images/",
            isTest = True,
            frac = 0.10
        )
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

Index(['Unnamed: 0', 'id', 'Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration',
       'Mass', 'Nodule', 'Pleural Thickening', 'Pneumonia', 'Pneumothorax',
       'Pneumoperitoneum', 'Pneumomediastinum', 'Subcutaneous Emphysema',
       'Tortuous Aorta', 'Calcification of the Aorta', 'No Finding', 'subj_id',
       'Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width',
       'Height]', 'OriginalImagePixelSpacing[x', 'y]'],
      dtype='object')
7851


In [8]:
device

device(type='mps')

In [1]:
# train
NUM_EPOCHS = 10
for epoch in range(NUM_EPOCHS):
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    print('Epoch [%d/%d]'% (epoch+1, NUM_EPOCHS))
    model.train()
    for inputs, _, targets in tqdm(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        print(inputs.shape, targets.shape)
        # forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(inputs)
        print(targets[0])
        print(outputs[0])
        
        # if task == 'multi-label, binary-class':
        targets = targets.to(torch.float32)
        loss = criterion(outputs, targets)
        # else:
        #     targets = targets.squeeze().long()
        #     loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()

        print(loss)

Epoch [1/10]


NameError: name 'model' is not defined